In [1]:
import os
import math
import pandas as pd
import numpy as np
import albumentations as A
import cv2 as cv

In [ ]:
DATASET_PATH = "F:\\CASME2\\Cropped"
OUTPUT_DATASET_PATH = "F:\\CASME2\\test"

In [ ]:
def frame_path(subject_number, expression_id, frame_number, dataset_path=DATASET_PATH):
    return os.path.join(dataset_path, 'sub{:02d}'.format(subject_number), expression_id, f'reg_img{frame_number}.jpg')

In [ ]:
objective_class = pd.read_excel("F:\\CASME2\\CASME2-ObjectiveClasses.xlsx") # This contains label using objective class 1-7
frame_info = pd.read_excel("F:\\CASME2\\CASME2-coding-20140508.xlsx", usecols=(0,1,3,4,5), dtype={'OnsetFrame': 'Int32', 'ApexFrame': 'Int32', 'OffsetFrame': 'Int32', 'Subject': 'Int32'}, na_values={'ApexFrame': '/'}).dropna()

objective_class.set_index(['Subject', 'Filename'])
frame_info.set_index(['Subject', 'Filename'])

In [ ]:
label = pd.merge(frame_info, objective_class, on=['Subject', 'Filename'], how='inner')

In [ ]:
label.insert(3, 'OnsetApexFrame', ((label.OnsetFrame.to_numpy() + label.ApexFrame.to_numpy()) / 2).astype(np.int32))
label.insert(5, 'ApexOffsetFrame', ((label.ApexFrame.to_numpy() + label.OffsetFrame.to_numpy()) / 2).astype(np.int32))

In [ ]:
processed_label = []

In [ ]:
horizontal_flips = [False, True]
translations = [(0,0), (-2,-2), (-2,2), (2,-2), (2,2)]
rotations = [-10, -5, 0, 5, -10]
scales = [0.9, 1.0, 1.1]

def augment_and_save( image, on_save, counter, output_path=OUTPUT_DATASET_PATH):
    c = counter
    for horizontal_flip in horizontal_flips:
        for translation in translations:
            for rotation in rotations:
                for scale in scales:
                    transformer  = A.Compose([
                        A.HorizontalFlip(p=1.0 if horizontal_flip else 0.0),
                        A.Affine(
                            p=1.0,
                            translate_px={'x': translation[0], 'y': translation[1]},
                            scale=scale,
                            rotate=rotation,
                        )
                    ])
                    
                    augmented_image = transformer(image=image)['image']
                    filename = os.path.join(OUTPUT_DATASET_PATH, "{:09d}.jpg".format(c))
                    cv.imwrite(filename, augmented_image)
                    c += 1
                    
                    on_save("{:09d}".format(c), c)
    return c
 
def a(exp_class, exp_state):
    def cb(filename, i):
        processed_label.append({
            'Filename': filename,
            'ExpressionClass': exp_class,
            'ExpressionState': exp_state
        })
    return cb
    

In [ ]:
counter = 1
for _, row in label.iterrows():
    
    exp_class = row['Objective Class']
    
    onset_frame = cv.imread(frame_path(subject_number=row['Subject'], expression_id=row['Filename'], frame_number=row['OnsetFrame']))
    onsetapex_frame = cv.imread(frame_path(subject_number=row['Subject'], expression_id=row['Filename'], frame_number=row['OnsetApexFrame']))
    apex_frame = cv.imread(frame_path(subject_number=row['Subject'], expression_id=row['Filename'], frame_number=row['ApexFrame']))
    apexoffset_frame = cv.imread(frame_path(subject_number=row['Subject'], expression_id=row['Filename'], frame_number=row['ApexOffsetFrame']))
    offset_frame = cv.imread(frame_path(subject_number=row['Subject'], expression_id=row['Filename'], frame_number=row['OffsetFrame']))
    
    counter = augment_and_save(onset_frame, a(exp_class, 1), counter=counter)
    counter = augment_and_save(onsetapex_frame, a(exp_class, 2), counter=counter)
    counter = augment_and_save(apex_frame, a(exp_class, 3), counter=counter)
    counter = augment_and_save(apexoffset_frame, a(exp_class, 4), counter=counter)
    counter = augment_and_save(offset_frame, a(exp_class, 5), counter=counter)
    
processed_label.to_excel(os.path.join(OUTPUT_DATASET_PATH, "label.xlsx"))